In [4]:
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn import svm

In [5]:
df = pd.read_csv('falldeteciton.csv')
df.head()

,ACTIVITY,TIME,SL,EEG,BP,HR,CIRCLUATION
0,3,4722.92,4019.64,-1600.00,13,79,317
1,2,4059.12,2191.03,-1146.08,20,54,165
2,2,4773.56,2787.99,-1263.38,46,67,224
3,4,8271.27,9545.98,-2848.93,26,138,554
4,4,7102.16,14148.80,-2381.15,85,120,809


In [6]:
df2 = pd.concat([df, pd.get_dummies(df['ACTIVITY'], prefix='ACTIVITY')], axis=1)
df2 = df2.drop(['ACTIVITY'], axis=1)
df2.head()

,TIME,SL,EEG,BP,HR,CIRCLUATION,ACTIVITY_0,ACTIVITY_1,ACTIVITY_2,ACTIVITY_3,ACTIVITY_4,ACTIVITY_5
0,4722.92,4019.64,-1600.00,13,79,317,0,0,0,1,0,0
1,4059.12,2191.03,-1146.08,20,54,165,0,0,1,0,0,0
2,4773.56,2787.99,-1263.38,46,67,224,0,0,1,0,0,0
3,8271.27,9545.98,-2848.93,26,138,554,0,0,0,0,1,0
4,7102.16,14148.80,-2381.15,85,120,809,0,0,0,0,1,0


Our Goal is to predict the various classes of fall rates. Let us start by scaling all the columns

In [15]:
from sklearn.preprocessing import StandardScaler
y = df.iloc[:,0]
x = df.iloc[:,1:]
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=42)

## SVM

In [8]:
print('Lets Try some ploynomial kernels')
for i in range(10):
    clf = svm.SVC(kernel='poly', degree=i)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(' For degree ', i+1, ' the score is ',score)

Lets Try some ploynomial kernels
 For degree  1  the score is  0.27128471162648765
 For degree  2  the score is  0.31644797070491304
 For degree  3  the score is  0.301495270064083
 For degree  4  the score is  0.2956972841013122
 For degree  5  the score is  0.3082087274946598
 For degree  6  the score is  0.31339639914555995
 For degree  7  the score is  0.3143118706133659
 For degree  8  the score is  0.32194079951174853
 For degree  9  the score is  0.3152273420811718
 For degree  10  the score is  0.3188892279523955


In [10]:
print('Now lets try the RBF Kernel')
clf = svm.SVC()
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print(' For RBF the score is ',score)

Now lets try the RBF Kernel
 For RBF the score is  0.3939578883124809


## MLP

In [16]:
y = df2.iloc[:,-6:]
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=42)

In [18]:
from keras.optimizers import RMSprop
from keras import layers
from keras import Input
from keras.models import Model

data_input = Input(shape=(X_train.shape[1],), dtype='float32', name='vals')
x = layers.Dense(units=30, activation='relu')(data_input)
x = layers.Dense(units=15, activation='relu')(x)
x = layers.Dense(units=25, activation='relu')(x)
x = layers.Dense(units=65, activation='relu')(x) 
x = layers.Dense(units=120, activation='relu')(x) 

fall_prediction = layers.Dense(6, activation='softmax', name='fall')(x)

model = Model(data_input,fall_prediction)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])


model.fit(X_train, y_train, validation_split=0.2, epochs=12, batch_size=10, verbose=0)


score = model.evaluate(X_test, y_test, verbose=0)


print('Test accuracy_fall:', score[-1])

Test accuracy_fall: 0.5135794878005981


In [ ]:
print('Our ML model is far better at pattern recognition')